<a href="https://colab.research.google.com/github/ahzaidy/Programs/blob/main/CPSC_5440_HW21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, TensorDataset
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle
from google.colab import drive

# Mount Google Drive
drive.mount("/content/drive")

# Load CIFAR-100 dataset from Google Drive
with open('/content/drive/My Drive/train', 'rb') as file:
    train_dict = pickle.load(file, encoding='bytes')

with open('/content/drive/My Drive/test', 'rb') as file:
    test_dict = pickle.load(file, encoding='bytes')

# Extract data and labels
train_data = torch.tensor(train_dict[b'data'], dtype=torch.float32).reshape(-1, 3, 32, 32) / 255.0
train_labels = torch.tensor(train_dict[b'fine_labels'], dtype=torch.long)
test_data = torch.tensor(test_dict[b'data'], dtype=torch.float32).reshape(-1, 3, 32, 32) / 255.0
test_labels = torch.tensor(test_dict[b'fine_labels'], dtype=torch.long)

# Create datasets
train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)

class CIFAR100Net(nn.Module):
    def __init__(self, units, hidden_activation, output_activation):
        super(CIFAR100Net, self).__init__()
        self.hidden_activation = getattr(nn, hidden_activation)()
        self.output_activation = getattr(nn, output_activation)()
        self.fc1 = nn.Linear(3 * 32 * 32, units)
        self.hidden_layers = nn.ModuleList([nn.Linear(units, units) for _ in range(4)])
        self.output_layer = nn.Linear(units, 100)

    def forward(self, x):
        x = x.view(-1, 3 * 32 * 32)
        x = self.hidden_activation(self.fc1(x))
        for layer in self.hidden_layers:
            x = self.hidden_activation(layer(x))
        x = self.output_activation(self.output_layer(x))
        return x

def train_cifar100(config, checkpoint_dir=None):
    transform = transforms.Compose([transforms.Normalize((0.5,), (0.5,))])

    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(train_subset, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=config['batch_size'], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CIFAR100Net(config['units'], config['hidden_activation'], config['output_activation']).to(device)

    criterion = getattr(nn, config['loss'])()
    optimizer = getattr(optim, config['optimizer'])(model.parameters(), lr=config['lr'])

    for epoch in range(10):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        val_loss = 0.0
        correct = 0
        total = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = correct / total
        tune.report(loss=val_loss, accuracy=val_accuracy)

config = {
    'units': tune.choice([120, 240]),
    'hidden_activation': tune.choice(['ReLU', 'Sigmoid']),
    'output_activation': tune.choice(['Softmax', 'Sigmoid']),
    'loss': tune.choice(['CrossEntropyLoss', 'MSELoss']),
    'optimizer': tune.choice(['Adam', 'Adagrad']),
    'batch_size': tune.choice([1000, 2000]),
    'lr': tune.loguniform(1e-4, 1e-1)
}

ray.init(ignore_reinit_error=True)
scheduler = ASHAScheduler(
    metric='accuracy',
    mode='max',
    max_t=10,
    grace_period=1,
    reduction_factor=2
)

analysis = tune.run(
    train_cifar100,
    config=config,
    num_samples=10,
    scheduler=scheduler,
    resources_per_trial={'cpu': 2, 'gpu': 1}
)

df = analysis.results_df
best_trial = analysis.get_best_trial('accuracy', 'max', 'last')
best_config = best_trial.config
best_accuracy = best_trial.last_result['accuracy']

print(f'Best Hyperparameters: {best_config}')
print(f'Best Accuracy: {best_accuracy:.4f}')

plt.figure(figsize=(10, 5))
sns.lineplot(data=df, x='training_iteration', y='accuracy', hue='hidden_activation')
plt.xlabel('Training Iteration')
plt.ylabel('Validation Accuracy')
plt.title('Hyperparameter Tuning Results')
plt.grid()
plt.savefig("/content/drive/My Drive/hyperparameter_tuning_plot.png", dpi=300)
plt.show()

Mounted at /content/drive
